In [1]:
using Pkg
Pkg.activate("../..")
Pkg.instantiate()

  Activating project at `~/dev/projects/AWID`
Precompiling project...
    494.1 ms  ✓ AWID
  1 dependency successfully precompiled in 2 seconds. 205 already precompiled.


In [2]:
using JLD2
X_train = load("../../data/embeddings/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../../data/embeddings/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../../data/embeddings/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../../data/embeddings/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("../../data/embeddings/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("../../data/embeddings/imdb_dataset_prepared.jld2", "vocab")
nothing

println("X_train (type = ", typeof(X_train), "): size = ", size(X_train), ", first few values = ", X_train[1:5], " min = ", minimum(X_train), "max = ", maximum(X_train))
println("y_train (type = ", typeof(y_train), "): size = ", size(y_train), ", first few values = ", y_train[1:5])
println("X_test (type = ", typeof(X_test), "): size = ", size(X_test), ", first few values = ", X_test[1:5])
println("y_test (type = ", typeof(y_test), "): size = ", size(y_test), ", first few values = ", y_test[1:5])
println("embeddings (type = ", typeof(embeddings), "): size = ", size(embeddings), ", first few values = ", embeddings[1:5,1:5])
println("vocab: length = ", length(vocab), ", first few values = ", vocab[1:5])


embedding_dim = size(embeddings, 1)



X_train (type = LinearAlgebra.Adjoint{Int64, Matrix{Int64}}): size = (130, 40000), first few values = [9204, 9520, 3493, 3550, 965] min = 1max = 12837
y_train (type = BitMatrix): size = (1, 40000), first few values = Bool[1, 1, 0, 0, 1]
X_test (type = LinearAlgebra.Adjoint{Int64, Matrix{Int64}}): size = (130, 10000), first few values = [3261, 286, 8278, 2453, 12585]
y_test (type = BitMatrix): size = (1, 10000), first few values = Bool[1, 1, 0, 1, 0]
embeddings (type = Matrix{Float32}): size = (50, 12837), first few values = Float32[0.014323 -0.58014 0.27137 0.68397 0.11369; -0.74624 -1.1316 0.61347 -0.68729 0.53461; 0.35701 0.44189 -0.52498 0.8797 1.2828; 0.75488 -0.048199 -0.7617 -0.35249 -0.61803; 0.11551 -0.11754 0.37252 0.82288 0.064278]
vocab: length = 12837, first few values = ["dev", "dumber", "henry", "abducted", "rises"]


50

In [3]:
using AWID.NeuralNetwork, AWID.AutoDiff

model = Chain(
    Embedding(length(vocab), embedding_dim),
    Conv((3,), embedding_dim, 8, relu),
    MaxPool((8,)),
    Flatten(),
    Dense(128, 1, sigmoid)
)

# add Glove embeddings to Embedding layer
model.layers[1].W.output .= embeddings;

### Należy przepisać kod tak by w 'layers' był setup operatorów a działania na danych w operatorach

In [4]:
using AWID.NeuralNetwork, AWID.AutoDiff, Printf

batch_size = 64

train_on_batch, test_loss_and_accuracy = setup_training_functions(
    model=model,
    loss_fn=binary_crossentropy,
    accuracy_fn=accuracy,
    optimizer=Adam(),
    x_test=X_test,
    y_test=y_test,
    batch_size=batch_size,
)

epochs = 5
for epoch in 1:epochs
    epoch_total_loss = 0.0f0
    epoch_total_acc = 0.0f0
    num_processed_batches = 0

    epoch_batches = get_epoch_batches(X_train, y_train, batch_size=batch_size, do_shuffle=true)

    t = @elapsed begin
        for (x_batch, y_batch) in epoch_batches
            println("x_batch: ", size(x_batch))
            println("y_batch: ", size(y_batch))
            batch_loss, batch_acc = train_on_batch(x_batch, y_batch)

            epoch_total_loss += batch_loss
            epoch_total_acc += batch_acc
            num_processed_batches += 1
        end

        train_loss = epoch_total_loss / num_processed_batches
        train_acc = epoch_total_acc / num_processed_batches

        test_loss, test_acc = test_loss_and_accuracy()
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)",
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

layer: Conv


MethodError: MethodError: no method matching size(::Nothing)
The function `size` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  size(!Matched::BitVector)
   @ Base bitarray.jl:104
  size(!Matched::BitVector, !Matched::Integer)
   @ Base bitarray.jl:107
  size(!Matched::Core.Compiler.StmtRange)
   @ Base show.jl:2820
  ...
